#### Weighted Regression in =python=



The fact that $T$ and $u$ are &ldquo;independent&rdquo; (or at least
orthogonal) variables means that if we want to compute a
&ldquo;classical&rdquo; regression we&rsquo;d do it something like this:



##### Define independent random variables



In [1]:
%matplotlib inline
import numpy as np
from scipy.stats import multivariate_normal
np.random.seed(seed=90210)
k = 3 # Number of observables in T

mu = [0]*k
Sigma=[[1,0.5,0],
       [0.5,2,0],
       [0,0,3]]

T = multivariate_normal(mu,Sigma)

U = multivariate_normal(cov=0.2)

##### Define =X=



Recall that $X$ can depend on $T$ and $u$.  This dependence needn&rsquo;t be
linear!  For example, suppose $X=T^3D + u$, where $D$ is an
$\ell\times k$ matrix.



##### Construct Sample



To construct a sample of observables $(y,X,T)$ we just use the regression equation,
      plus an assumption about the value of $\beta$:



In [2]:
beta1 = [1/2,1]
beta2 = [2,4]
beta = np.reshape([beta1,beta2],(2,2))
m = 2

D = np.random.random(size=(3,2)) # Generate random 3x2 matrix

N=1000 # Sample size

# Now: Transform rvs into a sample
t = T.rvs(N)

u = U.rvs(N*m) # Replace u with a sample
u = np.reshape(u,(m,N)).T #Ensures the first N entries match those from weighted_regression.py


X = (t**3)@D  # Note use of ** operator for exponentiation

y = X@beta + u # Note use of @ operator for matrix multiplication

In [11]:
np.shape(u)

(1000, 2)

##### Turn to estimation



So, we now have data on *realizations* $(y,X,T)$.  Now forget
     that we know $\beta$ and let&rsquo;s estimate it, using weighted least
     squares.  As a numerical matter it&rsquo;s better to avoid explicitly
     inverting the $(T^T X)$ matrix; instead we can solve the &ldquo;normal&rdquo;
     equations

\begin{align*}
   X'y &= X' X b + X' u\\
   \mbox{E}(T'u) = 0
\end{align*}



##### Numerical solution



In the classical case we were trying to solve a linear system that
 took the form $Ab=0$, with $A$ a square matrix.  In the present case
 we&rsquo;re also trying to solve a linear system, but with a matrix $A$
 that may have more rows than columns.  Provided the rows are linearly
 independent, this implies that we have an **overidentified** system of
 equations.  We&rsquo;ll return to the implications of this later, but for
 now this also calls for a different numerical approach, using
 `np.linalg.lstsq` instead of `np.linalg.solve`.



In [3]:
from scipy.linalg import inv, sqrtm

b = np.linalg.lstsq(t.T@X,t.T@y)[0] # lstsqs returns several results; we pick the first

e = y - X@b

print(b)

TXplus = np.linalg.pinv(t.T@X) # Moore-Penrose pseudo-inverse

# Covariance matrix of b
e_var = np.zeros(m)
for i in range(m):
    e_var[i] = (e[:,0]).var()
e_var = np.diag(e_var)
    
    
vb = e.var()*TXplus@t.T@t@TXplus.T  # u is known to be homoskedastic

print(vb)

[[0.49960835 0.99977111]
 [2.00014854 4.00106422]]
[[ 2.53460798e-06 -1.27951640e-06]
 [-1.27951640e-06  4.57959173e-06]]


/tmp/ipykernel_215/2214551263.py:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  b = np.linalg.lstsq(t.T@X,t.T@y)[0] # lstsqs returns several results; we pick the first
